# Save sample data for testing

In [1]:
import boto3
import os
from tqdm import tqdm

s3 = boto3.client("s3")

# List all objects helper internal function
def list_all_objects(bucket, prefix):
    # Create a paginator for list_objects_v2
    paginator = s3.get_paginator('list_objects_v2')

    # Use the paginator to iterate through all pages
    all_objects = []
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        if 'Contents' in page:
            all_objects.extend(page['Contents'])

    return all_objects

bucket = 'udacity-deeplearning-project'
prefix = 'data'
sample_prefix = 'sample'

all_files = list_all_objects(bucket, prefix)

sample_files = [file_meta for file_meta in all_files if '001.Affenpinscher' in file_meta.get("Key")]

for file_meta in tqdm(sample_files):
    key = file_meta.get("Key")
    
    # Move the data from s3 to a different prefix
    dirname = os.path.dirname(key)
    if dirname:
        os.makedirs(dirname, exist_ok=True)

    s3.download_file(bucket, key, key)
    # s3.copy_object(
    #     Bucket=bucket,
    #     CopySource=f"{bucket}/{key}",
    #     Key=f"{sample_prefix}/{key}"
    # )

100%|██████████| 80/80 [00:08<00:00,  8.97it/s]


## Testing Estimator Locally Prior to Deployment to ECR

In [6]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
import sagemaker
import subprocess

subprocess.run(['sh', 'docker-build.sh'])

role=get_execution_role()

bucket = 'udacity-deeplearning-project'
sample_prefix = 'sample'

hyperparameters = {
    'num-classes': 133,
    'batch-size': 32,
    'lr': 0.005070970373087015
}

bucket = 'udacity-deeplearning-project'
s3_output_location = f"s3://{bucket}/outputs"

estimator=Estimator(
    image_uri='udacity-sagemaker-hpo',
    role=role,
    instance_count=1,
    instance_type='local',
    output_path=s3_output_location,
    hyperparameters=hyperparameters
)

model_inputs = {
    "train": "file://./data/test",
    "test": "file://./data/valid"
}

estimator.fit(inputs=model_inputs, logs=True ,wait=True)

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sha256:7ff4fe6551d548947f3183db86a68f3e09458ba39d737c3114b061ecef92b1e9
REPOSITORY                                                           TAG       IMAGE ID       CREATED         SIZE
598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo   latest    7ff4fe6551d5   7 minutes ago   3.92GB
udacity-sagemaker-hpo                                                latest    7ff4fe6551d5   7 minutes ago   3.92GB


INFO:sagemaker:Creating training-job with name: udacity-sagemaker-hpo-2024-11-20-14-52-59-961
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found using Docker Compose CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.i

 Container jxs1geaoc3-algo-1-hswu6  Creating
 Container jxs1geaoc3-algo-1-hswu6  Created
Attaching to jxs1geaoc3-algo-1-hswu6
jxs1geaoc3-algo-1-hswu6  | sed: can't read changehostname.c: No such file or directory
jxs1geaoc3-algo-1-hswu6  | gcc: error: changehostname.c: No such file or directory
jxs1geaoc3-algo-1-hswu6  | gcc: fatal error: no input files
jxs1geaoc3-algo-1-hswu6  | compilation terminated.
jxs1geaoc3-algo-1-hswu6  | gcc: error: changehostname.o: No such file or directory
jxs1geaoc3-algo-1-hswu6  | ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
jxs1geaoc3-algo-1-hswu6  | 2024-11-20 14:53:00,856 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
jxs1geaoc3-algo-1-hswu6  | 2024-11-20 14:53:00,859 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
jxs1geaoc3-algo-1-hswu6  | 2024-11-20 14:53:00,872 sagemaker_pytorch_container.tra

INFO:sagemaker.local.image:===== Job Complete =====


jxs1geaoc3-algo-1-hswu6 exited with code 0
Aborting on container exit...
 Container jxs1geaoc3-algo-1-hswu6  Stopping
 Container jxs1geaoc3-algo-1-hswu6  Stopped


## Testing Deployed Estimator

In [2]:
!sh push-container.sh

push-container.sh: line 1: fg: no job control
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (1/1) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 2B                                         0.0s
ERROR: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory
The push refers to repository [598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo]

bf18a086: Preparing 
f106088b: Preparing 
95045e04: Preparing 
e9708ca1: Preparing 
8f6060c6: Preparing 
a3c12226: Preparing 
62daa95e: Preparing 
8fe1cb59: Preparing 
061a5b0d: Preparing 
3ff1bf08:

In [3]:
model_inputs = {
    "train": sagemaker.inputs.TrainingInput(
        s3_data=f"s3://{bucket}/{sample_prefix}/data/train/",
        content_type="application/x-image"
    ),
    "test": sagemaker.inputs.TrainingInput(
        s3_data=f"s3://{bucket}/{sample_prefix}/data/valid/",
        content_type="application/x-image"
    )
}

In [5]:
hyperparameters = {
    'num-classes': 133,
    'batch-size': 32,
    'lr': 0.005070970373087015
}

In [4]:
estimator=Estimator(
    image_uri='598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo',
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters
)

estimator.fit(inputs=model_inputs, logs=True)

INFO:sagemaker:Creating training-job with name: udacity-sagemaker-hpo-2024-11-20-14-46-02-678


2024-11-20 14:46:04 Starting - Starting the training job
2024-11-20 14:46:04 Pending - Training job waiting for capacity......
2024-11-20 14:47:05 Pending - Preparing the instances for training......
2024-11-20 14:47:52 Downloading - Downloading input data...
2024-11-20 14:48:11 Downloading - Downloading the training image...
2024-11-20 14:49:03 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ign

# END OF NOTEBOOK